In [56]:
import tensorflow as tf

In [57]:
model = tf.keras.models.load_model("ce_446.5002951523832_11.90561973867756")

In [61]:
from arnaud_mechant import mechant_romain, mechant_arnaud



df_train, df_test = mechant_romain()
df_ = df_test[0]

df_train = df_train[0]


In [62]:
def feature_engineering(df_):
    date_time = df_["Date et heure de comptage"]
    timestamp_s = date_time.map(pd.Timestamp.timestamp)

    day = 24 * 60 * 60
    week = 7 * day

    import numpy as np

    df_["daySin"] = np.sin(timestamp_s * (2 * np.pi / day))
    df_["weekSin"] = np.sin(timestamp_s * (2 * np.pi / week))
    df_["Durée avant les prochaines vacances scolaires"] = df_[
        "Durée avant les prochaines vacances scolaires"
    ].dt.days
    df_ = df_[
        [
            "Débit horaire",
            "Taux d'occupation",
            "Etat trafic",
            "daySin",
            "weekSin",
            "humidity",
            "Durée avant les prochaines vacances scolaires",
        ]
    ]
    df_["filter"] = df_.isna().any(axis=1)
    return df_

df_ = feature_engineering(df_)
df_train = feature_engineering(df_train)

/var/folders/m8/sb3p17zj07s4h28sd6njtsbw0000gn/T/ipykernel_46996/567239433.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["filter"] = df_.isna().any(axis=1)
/var/folders/m8/sb3p17zj07s4h28sd6njtsbw0000gn/T/ipykernel_46996/567239433.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["filter"] = df_.isna().any(axis=1)


In [63]:
feature_columns = [
    "Débit horaire",
    "Taux d'occupation",
    "Etat trafic",
    "daySin",
    "weekSin",
    "humidity",
    "Durée avant les prochaines vacances scolaires",
]

df_ = df_[feature_columns]
df_train = df_train[feature_columns]

In [65]:
df_mean, df_std = df_train.mean(), df_train.std()

df_predict = (df_-df_mean)/df_std
df_train = (df_train-df_mean)/df_std


In [73]:
WIN_LENGTH = 24 * 5

    
input_width=int(2.5 * WIN_LENGTH)
label_width=WIN_LENGTH

missing = len(df_predict) - input_width - label_width
df_data = pd.concat([df_train[-299:].astype(float).interpolate(), df_predict], axis=0)

In [93]:
WIN_LENGTH = 24 * 5
    
input_width=int(2.5 * WIN_LENGTH)
label_width=WIN_LENGTH

y_target = df_data[-label_width:][["Débit horaire", "Taux d'occupation"]].to_numpy().reshape(1,120,2)

x = df_data[-input_width-label_width:-label_width].to_numpy().reshape(1,300,7)
y_pred = model.predict(np.random.rand(1, input_width, len(feature_columns)))

In [94]:
y_pred = model.predict(x.reshape(1,300,7))

m_1 = tf.keras.metrics.RootMeanSquaredError()
m_1.update_state(y_target[:,:,0], y_pred[:,:,0])

m_2 = tf.keras.metrics.RootMeanSquaredError()
m_2.update_state(y_target[:,:,1], y_pred[:,:,1])


std_1 = df_std["Débit horaire"]
std_2 = df_std["Taux d'occupation"]

kpi_1 = m_1.result().numpy() * std_1
kpi_2 = m_2.result().numpy() * std_2

kpi_1, kpi_2



(159.86642531099707, 11.758476464255162)